In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [2]:
from prefect import Flow, Parameter, unmapped
import pandas as pd
from prefect.engine.executors import DaskExecutor
from ml_flow import data_cleaning_flow
from tinydb import TinyDB

In [5]:
input_df = pd.read_csv("../data/titanic/train.csv")
test= pd.read_csv("../data/titanic/test.csv")

In [6]:
executor = DaskExecutor()
data_cleaner = data_cleaning_flow.run(
    input_data= input_df, 
    problem="binary classification", 
    target = "Survived", 
    features = "infer",
    executor=executor
)

[2020-04-13 02:39:40,272] INFO - prefect.FlowRunner | Beginning Flow run for 'data_cleaning'
[2020-04-13 02:39:40,277] INFO - prefect.FlowRunner | Starting flow run.
[2020-04-13 02:39:40,540] INFO - prefect.TaskRunner | Task 'input_data': Starting task run...
[2020-04-13 02:39:40,548] INFO - prefect.TaskRunner | Task 'input_data': finished task run for task with final state: 'Success'
[2020-04-13 02:39:40,668] INFO - prefect.TaskRunner | Task 'target': Starting task run...
[2020-04-13 02:39:40,703] INFO - prefect.TaskRunner | Task 'target': finished task run for task with final state: 'Success'
[2020-04-13 02:39:40,733] INFO - prefect.TaskRunner | Task 'problem': Starting task run...
[2020-04-13 02:39:40,736] INFO - prefect.TaskRunner | Task 'features': Starting task run...
[2020-04-13 02:39:40,772] INFO - prefect.TaskRunner | Task 'problem': finished task run for task with final state: 'Success'
[2020-04-13 02:39:40,774] INFO - prefect.TaskRunner | Task 'extract_nan_features': Startin

[2020-04-13 02:39:41,469] INFO - prefect.TaskRunner | Task 'fit_target_encoder': finished task run for task with final state: 'Success'
[2020-04-13 02:39:41,518] INFO - prefect.TaskRunner | Task 'target_encoder_transform': Starting task run...
[2020-04-13 02:39:41,519] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': Starting task run...
[2020-04-13 02:39:41,531] INFO - prefect.TaskRunner | Task 'target_encoder_transform': Starting task run...
[2020-04-13 02:39:41,532] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': Starting task run...
[2020-04-13 02:39:41,545] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': finished task run for task with final state: 'Success'
[2020-04-13 02:39:41,566] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': finished task run for task with final state: 'Success'
[2020-04-13 02:39:41,577] INFO - prefect.TaskRunner | Task 'target_encoder_transform': finished task run for tas

In [6]:
with Flow("data_visualization") as flow:
    transformed_train_df = pd.read_feather("transformed_train.df")
    transformed_valid_df = pd.read_feather("transformed_valid.df")
    svd = fit_svd(transformed_train_df)
    svd_train = svd_transform(svd, transformed_train_df, "transformed_train_df",tinydb)
    svd_valid = svd_transform(svd, transformed_valid_df, "transformed_valid_df",tinydb)
    

NameError: name 'fit_svd' is not defined

In [4]:
from meta_model import MetaModel, meta_model_flow
from tinydb import TinyDB
tinydb = TinyDB("db.json")
meta_model = MetaModel(problem="regression", db=tinydb,)

In [ ]:
import cloudpickle
# d =cloudpickle.dumps(meta_model)
# cloudpickle.loads(d)
d = cloudpickle.dumps(meta_model.models[0])
cloudpickle.loads(d)

In [6]:
d

b'\x80\x05\x95\x9c\r\x00\x00\x00\x00\x00\x00]\x94(\x8c\nmeta_model\x94\x8c\x05Model\x94\x93\x94)\x81\x94}\x94(\x8c\x07problem\x94\x8c\nregression\x94\x8c\x05model\x94\x8c(sklearn.linear_model._coordinate_descent\x94\x8c\nElasticNet\x94\x93\x94)\x81\x94}\x94(\x8c\x05alpha\x94G?\xf0\x00\x00\x00\x00\x00\x00\x8c\x08l1_ratio\x94G?\xe0\x00\x00\x00\x00\x00\x00\x8c\rfit_intercept\x94\x88\x8c\tnormalize\x94\x89\x8c\nprecompute\x94\x89\x8c\x08max_iter\x94M\xe8\x03\x8c\x06copy_X\x94\x88\x8c\x03tol\x94G?\x1a6\xe2\xeb\x1cC-\x8c\nwarm_start\x94\x89\x8c\x08positive\x94\x89\x8c\x0crandom_state\x94N\x8c\tselection\x94\x8c\x06cyclic\x94\x8c\x10_sklearn_version\x94\x8c\x0c0.22.2.post1\x94ub\x8c\nparam_hash\x94\x8c)<sha256 HASH object @ 0x000001640BBA9C70>\x94\x8c\x03uid\x94\x8c\x04uuid\x94\x8c\x04UUID\x94\x93\x94)\x81\x94}\x94\x8c\x03int\x94\x8a\x10a\xce\xf7\x87#\x1e)\xa9GEO\xcb.\x0b\xd4/sb\x8c\x02db\x94\x8c\x0ftinydb.database\x94\x8c\x06TinyDB\x94\x93\x94)\x81\x94}\x94(\x8c\n_cls_table\x94h\'\x8c\x05Tab

In [ ]:
from meta_model import MetaModel, meta_model_flow
tinydb = TinyDB("db.json")
executor = DaskExecutor()
meta_model = MetaModel(problem="regression", db=tinydb,)
meta_model_run = meta_model_flow.run(
    train_data = "transformed_train.df",
    valid_data = "transformed_valid.df",
    train_target = "survived",
    meta_model = meta_model,
    executor = executor,
    problem = "binary classification"
)

[2020-04-13 02:39:46,969] INFO - prefect.FlowRunner | Beginning Flow run for 'meta_model_flow'
[2020-04-13 02:39:46,973] INFO - prefect.FlowRunner | Starting flow run.


In [2]:
dir(meta_model)

NameError: name 'meta_model' is not defined

In [6]:
#flow.visualize(flow_state=flow_state)

In [86]:
from tinydb import TinyDB, Query
db = TinyDB("db.json")
db.all()

[]

In [ ]:
q = Query()
r = db.search(q.chunk == "svdname")

In [75]:
dir(q.chunk

['__and__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_generate_test',
 '_path',
 '_prepare_test',
 '_test',
 'all',
 'any',
 'exists',
 'hashval',
 'matches',
 'one_of',
 'search',
 'test']

In [119]:
flow._sorted_tasks()[38]

typing.Any

In [110]:
dir(flow._sorted_tasks()[38])

['__add__',
 '__and__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__mifflin__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 'auto_generated',
 'bind',
 'cache_for',
 'cache_key',
 'cache_validator',
 'checkpoint',
 'copy',
 'inputs',
 'is_equal',
 'is_not_equal',
 'log_stdout',
 'logger',
 'map',
 'max_retries',
 'name',
 'not_',
 'or_',
 'outputs',
 'result_handler',
 'retry_delay',
 'run',
 'serialize',
 'set_dependencies',
 'set_